---
title: "pheno heads up (blast) table" 
format: html
execute:
    eval: true
jupyter: IRkernel
---

In [1]:
library(jsonlite)
library(DT)
library(IRdisplay)
library(htmlwidgets)

In [2]:
config_dir <- "../phenoRankeR/inst/extdata/config"
json_data <- fromJSON(file.path(config_dir, "pheno_blast_col_colors.json"))

In [11]:
user <- "pheno.ranker@playground.com"
run_id <- "20241006015118"

user_dir <- file.path("../data/user_data", user)
out_dir <- file.path(user_dir, "output/rankedPatients", run_id)
file_suffix <- "_alignment.target.csv"

file_path <- file.path(
  out_dir,
  paste0(run_id, file_suffix)
)
print(file_path)

alignment_df <- read.table(
  file_path,
  header = TRUE,
  sep = ";",
  row.names = NULL
)
head(alignment_df)

[1] "../data/user_data/pheno.ranker@playground.com/output/rankedPatients/20241006015118/20241006015118_alignment.target.csv"


,id,ref,indicator,tar,weight,hamming.distance,json.path,label
,<chr>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,HG00100,0,,0,1,0,diseases.ICD10:D50.diseaseCode.id.ICD10:D50,iron deficiency anaemia
2,HG00100,0,,0,1,0,diseases.ICD10:E06.diseaseCode.id.ICD10:E06,thyroiditis
3,HG00100,0,,0,1,0,diseases.ICD10:J40.diseaseCode.id.ICD10:J40,acute bronchitis
4,HG00100,0,,0,1,0,diseases.ICD10:J45.diseaseCode.id.ICD10:J45,asthma
5,HG00100,0,,0,1,0,diseases.ICD10:K02.diseaseCode.id.ICD10:K02,dental caries
6,HG00100,0,,0,1,0,ethnicity.id.NCIT:C16352,Black or Black British


In [12]:
pat_id <- "HG00100"

# alignment_df
filtered_df <- alignment_df[alignment_df[, 1] == pat_id, ]

# remove the first column
filtered_df <- filtered_df[, -1]
colnames(filtered_df) <- c(
  "Reference", "Indicator",
  "Target", "Weight",
  "Hamming Distance", "JSON Path", "Label"
)

filtered_df

,Reference,Indicator,Target,Weight,Hamming Distance,JSON Path,Label
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,0,,0,1,0,diseases.ICD10:D50.diseaseCode.id.ICD10:D50,iron deficiency anaemia
2,0,,0,1,0,diseases.ICD10:E06.diseaseCode.id.ICD10:E06,thyroiditis
3,0,,0,1,0,diseases.ICD10:J40.diseaseCode.id.ICD10:J40,acute bronchitis
4,0,,0,1,0,diseases.ICD10:J45.diseaseCode.id.ICD10:J45,asthma
5,0,,0,1,0,diseases.ICD10:K02.diseaseCode.id.ICD10:K02,dental caries
6,0,,0,1,0,ethnicity.id.NCIT:C16352,Black or Black British
7,0,,0,1,0,ethnicity.id.NCIT:C41260,Asian or Asian British
8,0,,0,1,0,ethnicity.id.NCIT:C41261,British
9,0,,0,1,0,ethnicity.id.NCIT:C42331,African


In [4]:
top_level_row <- bin_df[1,]

# in the top level row remove everything after the first dot
top_level_row <- gsub("\\..*", "", top_level_row)
top_level_row[1] <- "top_level"

# add the top level row to the data frame
bin_df2 <- rbind(top_level_row, bin_df)

input_format <- "bff"
# json_data

color_map <- json_data[[input_format]]

# add a new row containing the colors
color_row <- unlist(sapply(
  bin_df2[1,],
  function(x) color_map[[x]]
))
color_row <- c("background-color", color_row)

# add the color row to the data frame
bin_df2 <- rbind(color_row, bin_df2)
bin_df2

ERROR: Error: object 'bin_df' not found


In [28]:
# extra header row containing the top level value
# while its cells are merged by top level

top_level_to_count <- table(top_level_row[-1])
top_level_to_count

top_level_to_color <- setNames(
  unique(color_row[-1]),
  unique(top_level_row[-1])
)

top_level_to_color

header_row <- paste(
  sapply(names(top_level_to_count), function(name) {
    colspan_value <- top_level_to_count[[name]]
    color <- top_level_to_color[[name]]
    sprintf(
      '<th colspan="%d" style="background-color:%s;">%s</th>',
      colspan_value, color, name
    )
  }),
  collapse = ""  # Collapse into a single string
)
header_row


                 diseases                 ethnicity          geographicOrigin 
                        5                         5                         5 
                       id interventionsOrProcedures                  measures 
                       10                        10                         9 
                      sex 
                        2 

diseases                 ethnicity          geographicOrigin 
        "rgb(158, 1, 66)"        "rgb(213, 62, 79)"       "rgb(253, 174, 97)" 
                       id interventionsOrProcedures                  measures 
     "rgb(189, 195, 199)"      "rgb(255, 255, 191)"      "rgb(171, 221, 164)" 
                      sex 
       "rgb(94, 79, 162)"

[1] "<th colspan=\"5\" style=\"background-color:rgb(158, 1, 66);\">diseases</th><th colspan=\"5\" style=\"background-color:rgb(213, 62, 79);\">ethnicity</th><th colspan=\"5\" style=\"background-color:rgb(253, 174, 97);\">geographicOrigin</th><th colspan=\"10\" style=\"background-color:rgb(189, 195, 199);\">id</th><th colspan=\"10\" style=\"background-color:rgb(255, 255, 191);\">interventionsOrProcedures</th><th colspan=\"9\" style=\"background-color:rgb(171, 221, 164);\">measures</th><th colspan=\"2\" style=\"background-color:rgb(94, 79, 162);\">sex</th>"

In [29]:
# JavaScript to prepend the header row
headerCallback <- JS(
  "function(thead, data, start, end, display) {",
  sprintf("$(thead).closest('thead').prepend('<tr><th></th>%s</tr>');", header_row),
  "}"
)


# JavaScript to color the columns based on the values in the 2nd row of the data
initComplete <- JS(
  "function(settings, json) {",
  "$('tbody tr').each(function() {",
    # Loop over each column to set background color based on the value in the first row (background-color)
    "for (var i = 0; i < $('tbody tr:eq(0) td').length; i++) {",
      "var color = $('tbody tr:eq(0)').find('td:eq(' + i + ')').text();",
      "console.log(color);",

      # Apply background color to the cell
      "$('tbody tr').find('td:eq(' + i + ')').css('background-color', color);",
      "$('thead th.sorting').eq(i).css('background-color', color);",  # Apply background color to the header

      # hide the row with the colors
      "$('tbody tr').eq(0).hide();",
    "}",
  "});",
  "}"
)

# remove row 2 and 3
bin_df3 <- bin_df2[-c(2, 3),]

dt <- datatable(
  bin_df3,
  rownames = FALSE,
  escape = FALSE,
  options = list(
    headerCallback = headerCallback,
    initComplete = initComplete
  )
)

saved_wiget_path <- "notebooks/saved_widgets"
widget_fn <- "binary_representation.html"

saveWidget(
  dt,
  file.path(
    saved_wiget_path,
    widget_fn
  )
)


In [30]:
# make sure that vscode is running the live server
live_server_path <- "http://localhost:5500/jupyrenv/notebooks/saved_widgets"
iframe_src <- paste0(
  live_server_path,
  "/",
  widget_fn
)

display_html(
  paste0(
    "<iframe src='",
    iframe_src,
    "' width='100%' height='500px'></iframe>"
  )
)